#  Απαλλακτική Εργασία ΤΕΔε part 2 (do stuff)

In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
from sklearn.model_selection import GridSearchCV, cross_val_score

In [3]:
#import statistics for standard deviation 
from statistics import stdev

Create our dataframes

In [4]:
test_name = r'./clean_test.csv'
train_name = r'./clean_train.csv'

Train_df1 = pd.read_csv(train_name, sep=',', usecols=['Insult','Comment'])
Test_df1 = pd.read_csv(test_name, sep=',', usecols=['id','Insult','Comment']) # with labels

Train_df = Train_df1['Comment'] #without the labels
Test_df = Test_df1[['id','Comment']] #without the labels

labels_train = Train_df1['Insult'].to_numpy()  #training labels
labels_test = Test_df1['Insult'].to_numpy() #test labels

An error popped up during count vectorizer: np.nan is an invalid document, expected byte or unicode string.

So we change them into 'str' as recommended here: https://stackoverflow.com/questions/39303912/tfidfvectorizer-in-scikit-learn-valueerror-np-nan-is-an-invalid-document

In [5]:
Train_df.values.astype('U')
Test_df.Comment.values.astype('U')

array(['like this if you are a tribe fan',
       'you re idiot                       ',
       'i am a woman babs  and the only war on women i see is coming from jackazzes like you   i don t need your protection or your ignorant rhetoric masquerading as representing my best interests   ',
       ...,
       'sweetie pie is looking very much like her cousin maggie e   what a beauty this little one is  love you all to pieces   ',
       'ball4real where are you with your miami g ayness',
       'man    if you are a 3 point shooter  you must love playing with wade and bron  '],
      dtype='<U1387')

Our scoring labels

In [6]:
scoring_labels=['accuracy','f1']

In [7]:
# accuracy score maybe not needed ==> balanced_accuracy and f1_score not needed

from sklearn.metrics import accuracy_score

In [8]:
def Compute_scores(prediction, labels_test, model, train, labels_train):
    #scores is a list of scores
    
    accuracy = accuracy_score(labels_test, prediction)
    
    f1_scores = cross_val_score(model, train, labels_train, cv=10, scoring='f1')
    f1_avg = np.mean(f1_scores)
    f1_stdev = stdev(f1_scores)
    
    
    #f1_scores = cross_val_score(mnb, counts_train, labels_train, cv=10, scoring='f1_macro')

    
    print("f1 is (mean : %f) and (standard deviation: %f)" % ( f1_avg, f1_stdev ) )
    print("accuracy is %f" % ( accuracy ) )
          
    #return [accuracy, [f1_avg,f1_stdev]]
    return [accuracy, f1_avg , f1_stdev ]

# Count Vectorizer()

Για κάθε πείραμα NaiveBayes, θα φτιάχνουμε καινούργιο CountVectorizer (με τις οποιες βελτιώσεις)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
#bow_vectorizer = CountVectorizer(max_df=1.0, min_df=1, max_features=1000, stop_words='english')

#counts_train = bow_vectorizer.fit_transform(Train_df.values.astype('U'))
#counts_test = bow_vectorizer.transform(Test_df.Comment.values.astype('U'))

# Naive Bayes

<b>ΣΗΜΑΝΤΙΚΟ</b>: Για τα ερωτήματα Naive Bayes θα χρησιμοποιήσουμε το μοντέλο <b>BernoulliNB</b> είναι σαν το MultinomialNB αλλά για Binary Classification (έχει και την  παράμετρο <b>alpha</b> για το Laplace smoothing)

Σημαντικό2: Δεν θα κάνω GridSearchCV για τα hyperparameters (για νβγάλω πιο ξεκάθαρα αποτελέσματα). Θα χρησιμοποιήσω:

1) <b> alpha = 1.0e-10 </b> (almost no smoothing, γιατί (=1) είναι Laplace smoothing και (<1) είναι Lidstone smoothing). Αν βαλώ 0.0 , λέει ότι είναι κάτω από minimum, οπότε βάζει αυτόματα την παραπάνω τιμή

2) binarize = 0.0 (default)

3) class_prior = None (default)

4) fit_prior = True (βοηθάει τα scores, αργεί λίγο στο training)

In [11]:
from sklearn.naive_bayes import BernoulliNB

# Naive Bayes1
Χωρίς βελτιώσεις

Count Vectorizer()

In [12]:
bow_vectorizer = CountVectorizer(max_df=1.0, min_df=1, max_features=1000)

counts_train = bow_vectorizer.fit_transform(Train_df.values.astype('U'))
counts_test = bow_vectorizer.transform(Test_df.Comment.values.astype('U'))

In [13]:
bNB1 = BernoulliNB(alpha=1.0e-10,fit_prior=True)

Train/Fit our model

In [14]:
bNB1.fit(counts_train,labels_train)

BernoulliNB(alpha=1e-10, binarize=0.0, class_prior=None, fit_prior=True)

Predict

In [15]:
bNB1_pred = bNB1.predict(counts_test)

Our Scores

In [16]:
bNB1_scores = Compute_scores(bNB1_pred, labels_test, bNB1, counts_train, labels_train)

f1 is (mean : 0.562336) and (standard deviation: 0.027070)
accuracy is 0.523490


# Naive Bayes2
Εδώ κάνουμε <b>lemmatization</b> στον Count vectorizer όπως περιγράφεται στο <b>6.2.3.10</b>: https://scikit-learn.org/stable/modules/feature_extraction.html

In [17]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 

In [18]:
#import nltk
#nltk.download('wordnet')

In [19]:
class LemmaTokenizer:
    def __init__(self):
         self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
         return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

Now for our CountVectorizer()

In [20]:
bow_vectorizer = CountVectorizer(max_df=1.0, min_df=1, max_features=1000, tokenizer=LemmaTokenizer()) #HERE

counts_train = bow_vectorizer.fit_transform(Train_df.values.astype('U'))
counts_test = bow_vectorizer.transform(Test_df.Comment.values.astype('U'))

In [21]:
bNB2 = BernoulliNB(alpha=1.0e-10,fit_prior=True)

Train/Fit our model

In [22]:
bNB2.fit(counts_train,labels_train)

BernoulliNB(alpha=1e-10, binarize=0.0, class_prior=None, fit_prior=True)

Predict

In [23]:
bNB2_pred = bNB2.predict(counts_test)

See our scores

In [24]:
bNB2_scores = Compute_scores(bNB2_pred, labels_test, bNB2, counts_train, labels_train)

f1 is (mean : 0.566353) and (standard deviation: 0.027171)
accuracy is 0.522148


# Naive Bayes3
Αφαιρούμε τα <b>stopwords</b>

In [25]:
bow_vectorizer = CountVectorizer(max_df=1.0, min_df=1, max_features=1000, stop_words='english') #HERE

counts_train = bow_vectorizer.fit_transform(Train_df.values.astype('U'))
counts_test = bow_vectorizer.transform(Test_df.Comment.values.astype('U'))

In [26]:
bNB3 = BernoulliNB(alpha=1.0e-10,fit_prior=True)

Train/Fit our model

In [27]:
bNB3.fit(counts_train,labels_train)

BernoulliNB(alpha=1e-10, binarize=0.0, class_prior=None, fit_prior=True)

Predict

In [28]:
bNB3_pred = bNB3.predict(counts_test)

See our scores

In [29]:
bNB3_scores = Compute_scores(bNB3_pred, labels_test, bNB3, counts_train, labels_train)

f1 is (mean : 0.592411) and (standard deviation: 0.029309)
accuracy is 0.666667


# Naive Bayes4
Χρησιμοποιούμε <b>bigrams</b>

In [30]:
bow_vectorizer = CountVectorizer(max_df=1.0, min_df=1, max_features=1000, ngram_range=(2,2)) #HERE

counts_train = bow_vectorizer.fit_transform(Train_df.values.astype('U'))
counts_test = bow_vectorizer.transform(Test_df.Comment.values.astype('U'))

In [31]:
bNB4 = BernoulliNB(alpha=1.0e-10,fit_prior=True)

Train/Fit out model

In [32]:
bNB4.fit(counts_train,labels_train)

BernoulliNB(alpha=1e-10, binarize=0.0, class_prior=None, fit_prior=True)

Predict

In [33]:
bNB4_pred = bNB4.predict(counts_test)

See our scores

In [34]:
bNB4_scores = Compute_scores(bNB4_pred, labels_test, bNB4, counts_train, labels_train)

f1 is (mean : 0.571800) and (standard deviation: 0.043224)
accuracy is 0.608949


# Naive Bayes5
Χρησιμοποιύμε <b> Laplace Smoothing </b>

In [35]:
bow_vectorizer = CountVectorizer(max_df=1.0, min_df=1, max_features=1000)

counts_train = bow_vectorizer.fit_transform(Train_df.values.astype('U'))
counts_test = bow_vectorizer.transform(Test_df.Comment.values.astype('U'))

In [36]:
bNB5 = BernoulliNB(alpha=1.0,fit_prior=True) #HERE alpha=1.0 => Laplace smoothing

Train/Fit out model

In [37]:
bNB5.fit(counts_train,labels_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

Predict

In [38]:
bNB5_pred = bNB5.predict(counts_test)

See our scores

In [39]:
bNB5_scores = Compute_scores(bNB5_pred, labels_test, bNB5, counts_train, labels_train)

f1 is (mean : 0.594627) and (standard deviation: 0.030683)
accuracy is 0.534228


# Naive Bayes Αποτελέσματα 
Θα τα βάλω σε ένα dataframe για να φαίνονται καλύτερα

In [40]:
list1 = [bNB1_scores,bNB2_scores,bNB3_scores,bNB4_scores,bNB5_scores] # a list of lists
index1 = ['NB baseline', 'NB lemmatization', 'NB stopwords', 'NB bigrams', 'NB Laplace Smoothing']
columns1 = ['Accuracy','F1 (avg)', 'F1 (τυπική απόκλιση)']

NB_df = pd.DataFrame( data=list1, index=index1, columns=columns1)
NB_df

,Accuracy,F1 (avg),F1 (τυπική απόκλιση)
NB baseline,0.523490,0.562336,0.027070
NB lemmatization,0.522148,0.566353,0.027171
NB stopwords,0.666667,0.592411,0.029309
NB bigrams,0.608949,0.571800,0.043224
NB Laplace Smoothing,0.534228,0.594627,0.030683


Παρατηρήσεις:

1) Τα καλύτερα αποτελέσματα έχουμε <b> αφαιρώντας τα stopwords</b> τόσο σε accuracy όσο και σε f1 (με μια σχετικά καλή διακύμανση τιμών f1, συγκριτικά με το minimum ==> baseline). Δείχνει πως το να αφαιρείς τα stopwords είναι αρκετά σημαντικό.

2) Ακολουθεί το <b> NB bigrams </b> ,αλλά εμφανίζει πολύ μεγαλύτερη διακύμανση  (0,43224 >> 0.29309)

3) Το lemmatization πιθανώς να λειτουργούσε καλύτερα, αν δεν αφαιρούσαμε τα σημεία στίξης/punctuation (που απαιτεί η εκφώνηση).

4) Τα <b> bigrams </b> είναι λογικό να δίνει καλύτερα αποτελέσματα, γιατί δίνει παραπάνω "context" στην πρόταση και γίνεται πιο εύκολα το classification.

# Part 2!
Θα ασχοληθούμε με <b> POS tagging </b> , TF-IDF και τα μοντέλα SVM, random decision forest

<b>TfidfVectorizer()</b>

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [42]:
tfidf_vectorizer = TfidfVectorizer(max_df=1.0, min_df=1, max_features=1000,stop_words='english')

tfidf_train = tfidf_vectorizer.fit_transform(Train_df.values.astype('U'))
tfidf_test = tfidf_vectorizer.transform(Test_df.Comment.values.astype('U'))

# POS tagging
Επιδή θα χρειαστούμε να υπολογίσουμε κάποια fractions, θα χρησιμοποιήσουμε <b>collections.Counter </b>

https://medium.com/@gianpaul.r/tokenization-and-parts-of-speech-pos-tagging-in-pythons-nltk-library-2d30f70af13b

Έχω προσθέσει και 2 <b> δικά μου fractions/features </b>

1) <b>Personal Pronouns</b>

2)<b> wh-adverb and wh-pronouns</b>

In [43]:
import nltk
from collections import Counter
from scipy import sparse

In [44]:
#import nltk
#nltk.download('averaged_perceptron_tagger')

Μας ενδιαφέρουν:

1) adverb = (RB,RBR,RBS)

2) verb = (VB,VBD,VBG,VBN.VBP,VBZ)

3) adjective = (JJ,JJR,JJS)

4) nouns = (NN,NNS,NNP,NNPS)

### Προσθέτω και 2 δικά μου

5) Personal Pronouns (PRP, PRP$): "Who do you think you are?" "F*ck you", "You idiot"

6) wh-pronoun and wh-adverbs (WP,WP$,WRB): "Whose fault do you think this is?"
"What are you doing, idiot."


In [45]:
def POS_tagging(Series,tfidf_array):
    
    #list1 = [] #list of lists
    
    adverb_list = [] #number for each sample
    verb_list = []
    adjective_list = []    
    noun_list = []
    
    #########
    personalp_list = []
    wh_list =[]
    
    for comment in Series:
    
        tokens = nltk.word_tokenize(str(comment))
        tag_list = nltk.pos_tag(tokens) #HERE, POS tagging
        word_num = len(tag_list) # because this list is [(word1,tag),(word2,tag),.....]
          
        if(word_num == 0): #empty sentence
            adverb_list.append(0)
            verb_list.append(0)
            adjective_list.append(0)
            noun_list.append(0)
            
            personalp_list.append(0)
            wh_list.append(0)
            continue
        
    
        counter = Counter()
        for x in tag_list: # count the frequency for each tag
            counter[x[1]] +=1
    
        
        #1) adverb (RB,RBR,RBS)
        sum_adverb = counter['RB'] + counter['RBR'] + counter['RBS'] + counter['RBS']
        fraction_adverb = sum_adverb / word_num
        adverb_list.append(fraction_adverb)
        
        # 2) verb (VB,VBD,VBG,VBN.VBP,VBZ)
        sum_verb = counter['VB'] + counter['VBD'] + counter['VBG'] + counter['VBN'] + counter['VBP'] + counter['VBZ']
        sum_verb += counter['MD'] #modal verbs
        fraction_verb = sum_verb / word_num
        verb_list.append(fraction_verb)
        
        # 3) adjective = (JJ,JJR,JJS)
        sum_adjective = counter['JJ'] + counter['JJR'] + counter['JJS']
        fraction_adjective = sum_adjective / word_num
        adjective_list.append(fraction_adjective)
        
        #4) Nouns (NN,NNS,NNP,NNPS)
        sum_noun = counter['NN'] + counter['NNS'] + counter['NNP'] + counter['NNPS']
        fraction_noun = sum_noun / word_num
        noun_list.append(fraction_noun)
        
        ###### The last 2 are mine
        
        # 5) Personal Pronoun
        sum_personalp = counter['PRP'] + counter['PRP$']
        fraction_personalp = sum_personalp / word_num
        personalp_list.append(fraction_personalp)

        # 6) wh-adverb and wh-pronoun
        sum_wh = counter['WP'] + counter['WP$'] + counter['WRB']
        fraction_wh = sum_wh / word_num
        wh_list.append(fraction_wh)
        
        
    #end of loop => perform concats
    #print(tfidf_array.shape,len(adverb_list),len(verb_list),len(adjective_list),len(noun_list))
    
    dense_array = tfidf_array.todense()
    
    adverb_array = np.array(adverb_list)
    verb_array = np.array(verb_list)
    adjective_array = np.array(adjective_list)
    noun_array = np.array(noun_list)
    
    personalp_array = np.array(personalp_list)
    wh_array = np.array(wh_list)
    
    
    
    #add these arrays as new columns to tfidf array
    new_array = np.column_stack((dense_array,adverb_array,verb_array,adjective_array,noun_array))
    new_array = np.column_stack((new_array,personalp_array,wh_array))
    
    #turn it back to sparse (tfidf was sparse)
    new_array = sparse.csr_matrix(new_array)
    
    return new_array

Create the compostite <b>train array </b>

In [46]:
pos_train = POS_tagging(Train_df,tfidf_train)
pos_test = POS_tagging(Test_df.Comment,tfidf_test)

# Support Vector Machines

Συγκεκριμένα:

## SVC 

In [48]:
from sklearn.svm import SVC

Compute <b> HyperParameters </b>

In [51]:
#C = [1.0,5.0,10.0]
#C = [1.0]

#C=[1.0,10.0,100.0,1000.0,5000.0] # 1.0
#C = [0.1,0.01,0.001,0.0001,0.00001] #0.1
#C = [0.05,0.5] #0.5
#C = [0.7 ,0.2] #0.7
#C = [0.8,0.9] #0.8
#C = [0.7,0.8] #0.8
#C = [0.85,0.89] #0.85
#C = [0.85,0.84,0.83,0.82,0.81] #0.82
#C = [0.815,0.816,0.817,0.818,0.819,0.82, 0.821,0.822,0.823,0.824,0.825,0.826,0.827,0.828,0.829] # 0.82
C = [0.82]




#kernel = ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'] #not precomputed ==> need square matrix
#kernel = ['linear', 'poly', 'rbf', 'sigmoid'] 
kernel = ['rbf'] #or sigmoid or linear



#gamma = ['scale','auto'] #scale
gamma = ['scale']

cache_size = [1000] # MB

#######################
#probability = [False,True]
probability = [False] #Too expensive/incositent values (for binary case) => gridsearch verified

#class_weight = ['balanced',None]
class_weight = ['balanced']

In [52]:
pgrid = {'cache_size': cache_size, 'kernel':kernel, 'gamma':gamma, 
         'probability':probability, 'class_weight':class_weight, 'C':C}

In [53]:
grid = GridSearchCV(estimator=SVC(), param_grid=pgrid, scoring=scoring_labels, refit='f1', n_jobs=-1, cv=10 )

In [54]:
grid.fit(pos_train,labels_train)

svc = grid.best_estimator_ #returns the optimized model (in regards to the parameters)
grid.best_params_

{'C': 0.82,
 'cache_size': 1000,
 'class_weight': 'balanced',
 'gamma': 'scale',
 'kernel': 'rbf',
 'probability': False}

Train/Fit our model

In [55]:
svc.fit(pos_train,labels_train)

SVC(C=0.82, break_ties=False, cache_size=1000, class_weight='balanced',
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma='scale',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

Model.predict() and accuracy of our prediction

In [56]:
svc_pred = svc.predict(pos_test)

Compute our scores

In [57]:
svc_scores = Compute_scores(svc_pred, labels_test, svc, pos_train, labels_train)

f1 is (mean : 0.652908) and (standard deviation: 0.022871)
accuracy is 0.683669


# Random Forests

In [58]:
from sklearn.ensemble import RandomForestClassifier

Compute HyperParameters

In [59]:
#max_depth = [None] #default
#max_depth = [10,20,30,40,50,60,70,80,90,100] #60
#max_depth = [55,65] #55
#max_depth=[51,52,53,54,55,56,57,58,59,60] #52
max_depth=[52]  # ==> OK





#max_features = ['sqrt','log2'] #sqrt ->31.7
#max_features =['sqrt']
#max_features = [30,50,100,300] #30
#max_features = [20,25,30,35,40,45] #35
#max_features=[30,31,32,33,34,35,36,37,38,39] #35
max_features = [35] # =====> OK



#min_samples_leaf=[1] #default
#min_samples_leaf=[1,5,10,20,50,100] #5
#min_samples_leaf=[2,3,4,5,6,7,8,9] # 3
min_samples_leaf = [3] # ===> OK



#min_samples_split=[2] #default
#min_samples_split=[2,5,10,30,50,100] #50
#min_samples_split = [40,50,60,70,80,90] # 90
#min_samples_split =[85,95] #95
#min_samples_split=[90,91,92,93,94,95,96,97,98,99] #93
min_samples_split=[93] # ==> OK



#n_estimators = [100] #default
#n_estimators = [10,50,100,150,200,250] #150 or 200(2) or 250 ==> 200
#n_estimators = [160,170,180,190,200] # 190
#n_estimators = [185,190,195] # 195
#n_estimators = [191,192,193,194,195,196,197,198,199] # 199
n_estimators = [199] #==> OK

#warm_start =[False,True] #False
warm_start = [False]


#oob_score=[True,False] #False
oob_score=[False]

n_jobs=[-1]

#criterion=['gini','entropy'] #entropy
criterion = ['entropy']

class_weight = ['balanced']

In [60]:
pgrid = {'bootstrap':[False], 'max_depth':max_depth, 'max_features':max_features, 'min_samples_leaf':min_samples_leaf, 
         'min_samples_split': min_samples_split, 'n_estimators':n_estimators,
        'class_weight':class_weight, 'criterion':criterion, 'n_jobs':n_jobs,
        'oob_score':oob_score, 'warm_start':warm_start}



In [61]:
grid = GridSearchCV(estimator=RandomForestClassifier(), param_grid=pgrid, scoring=scoring_labels, refit='f1', cv=10, n_jobs=-1)
grid.fit(pos_train,labels_train)

rfc = grid.best_estimator_ #returns the optimized model (in regards to the parameters)
grid.best_params_

{'bootstrap': False,
 'class_weight': 'balanced',
 'criterion': 'entropy',
 'max_depth': 52,
 'max_features': 35,
 'min_samples_leaf': 3,
 'min_samples_split': 93,
 'n_estimators': 199,
 'n_jobs': -1,
 'oob_score': False,
 'warm_start': False}

Train/Fit our model

In [62]:
rfc.fit(pos_train,labels_train)

RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight='balanced',
                       criterion='entropy', max_depth=52, max_features=35,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=93,
                       min_weight_fraction_leaf=0.0, n_estimators=199,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

Model.predict() 

In [63]:
rfc_pred = rfc.predict(pos_test)

Compute our scores

In [64]:
rfc_scores = Compute_scores(rfc_pred, labels_test, rfc, pos_train, labels_train)

f1 is (mean : 0.642633) and (standard deviation: 0.030705)
accuracy is 0.680984


# Naive Bayes
Ας εξετάσουμε και αυτόν (με υπολογισμό των hyper parameters)

Υπολογίζουμε <b>hyper-parameters</b>

In [65]:
#alphas = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0] #1.0
#alphas = [1.0,10.0,100.0] #1.0
#alphas = [1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0] #2.0
#alphas = [1.5,2.5] #1.5
#alphas = [1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9] #1.6
#alphas = [1.55,1.65] #1.65
#alphas = [1.61,1.62,1.63,1.64,1.65,1.66,1.67,1.68,1.69] #1.61
alphas = [1.61] # ==> OK


pgrid = {'alpha': alphas, 'fit_prior': [True],'class_prior': [None] }
#Number of priors must match number of classes -> example: [0.1,0.1,0.1,0.1,0.6]

In [66]:
grid = GridSearchCV(estimator=BernoulliNB(), param_grid=pgrid, scoring=scoring_labels, refit='f1', cv=10, n_jobs=-1)
grid.fit(pos_train,labels_train)

bNB_final = grid.best_estimator_ #returns the optimized model (in regards to the parameters)
grid.best_params_

{'alpha': 1.61, 'class_prior': None, 'fit_prior': True}

Train our model and predict

In [67]:
bNB_final.fit(pos_train,labels_train)
bNB_final_pred = bNB_final.predict(pos_test)
bNB_final_scores = Compute_scores(bNB_final_pred, labels_test, bNB_final, pos_train, labels_train)

f1 is (mean : 0.654096) and (standard deviation: 0.032611)
accuracy is 0.686801


# Final Αποτελέσματα

In [68]:
list1 = [bNB1_scores,bNB2_scores,bNB3_scores,bNB4_scores,bNB5_scores,svc_scores,rfc_scores,bNB_final_scores] # a list of lists
index1 = ['NB baseline', 'NB lemmatization', 'NB stopwords', 'NB bigrams', 'NB Laplace Smoothing','SVM','Random Forest', 'NB final']
columns1 = ['Accuracy','F1 (avg)', 'F1 (τυπική απόκλιση)']

NB_df = pd.DataFrame( data=list1, index=index1, columns=columns1)
NB_df

,Accuracy,F1 (avg),F1 (τυπική απόκλιση)
NB baseline,0.523490,0.562336,0.027070
NB lemmatization,0.522148,0.566353,0.027171
NB stopwords,0.666667,0.592411,0.029309
NB bigrams,0.608949,0.571800,0.043224
NB Laplace Smoothing,0.534228,0.594627,0.030683
SVM,0.683669,0.652908,0.022871
Random Forest,0.680984,0.642633,0.030705
NB final,0.686801,0.654096,0.032611


Παρατηρήσεις:

1) Τα χαμηλά αποτελέσματα μάλλον οφείλονται στο data cleaning ή/και στα 2 features πoυ πρόσθεσα (δεν νομίζω να είναι τα hyperparameters)

2) Απο τα 3 τελευταία, το <b> NB final </b> (που είναι τπ πλήρως configured (Bernoulli)Naive Bayes πάνω στον σύνθετο πίνακα μας(tfidf+pos)) έχει τα καλύτερα αποτελέσματα και από θέμα <b> accuracy </b> και από <b> f1_score </b>.

3) Βέβαια ως προς <b> accuracy </b>, δεν απέχουν πολύ μεταξύ τους. Ως προς <b> f1 </b> είναι λίγο πιο αισθητή η διαφορά.

4) Ενδιαφέρον έχει το standard deviation. Δεν έχει μεγάλη διαφορά για το SVM,Random Forest αλλά είναι μεγαλύτερο του NB_final 
